# 산탄데르 제품 추천 경진대회

 Tabular 데이터를 다루는 캐글 경진대회에서의 머신러닝 파이프라인의 일반적인 순서는 다음과 같다.

1. 데이터 전처리
2. 피처 엔지니어링
3. 머신러닝 모델 학습
4. 테스트 데이터 예측 및 캐글 업로드

# 1. 데이터 전처리

- 제품 변수 결측 값 0으로 대체한다. 제품 보유 여부에 대한 정보가 없으므로 해당 제품을 보유하지 않다고 판단한다.
- 훈련 데이터와 테스트데이터를 통합한다. 훈련 데이터와 테스트 데이터는 날짜 변수로 쉽게 구분이 가능하다. 동일한 24개의 고객 변수를 공유하고 있으며 테스트 데이터에 없는 24개의 제품 변수는 0으로 채운다.
- 범주형, 수치형 데이터를 전처리한다. 범주형 데이터는 .factorize()를 통해 Label Encoding을 수행한다. 데이터 타입이 object 로 표현되는 수치형 데이터에서는 .unique()를 통해 특이값을 대체하거나 제거하고 정수형 데이터로 변환한다.
- 추후, 모델 학습에 사용할 변수 이름ㅇ르 features 리스트에 미리 담는다.


In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [2]:
np.random.seed(2018)

In [3]:
trn = pd.read_csv('/Users/lifesailor/.kaggle/santander/train_ver2.csv')
tst = pd.read_csv('/Users/lifesailor/.kaggle/santander/test_ver2.csv')

/anaconda3/envs/base-workspace/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/envs/base-workspace/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
trn.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [5]:
tst.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,2016-06-28,15889,F,ES,V,56,1995-01-16,0,256,1,...,N,N,KAT,N,1,28.0,MADRID,1,326124.90,01 - TOP
1,2016-06-28,1170544,N,ES,H,36,2013-08-28,0,34,1,...,N,NaN,KAT,N,1,3.0,ALICANTE,0,NA,02 - PARTICULARES
2,2016-06-28,1170545,N,ES,V,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,15.0,"CORUÑA, A",1,NA,03 - UNIVERSITARIO
3,2016-06-28,1170547,N,ES,H,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,8.0,BARCELONA,0,148402.98,03 - UNIVERSITARIO
4,2016-06-28,1170548,N,ES,H,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,7.0,"BALEARS, ILLES",0,106885.80,03 - UNIVERSITARIO


In [6]:
# 제품 변수
prods = trn.columns[24:].tolist()

# na 0으로 채우기
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# 24개 제품 중 하나도 보유하지 않는 고객 데이터 제거
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

In [7]:
# 훈련 데이터와 테스트 데이터를 통합한다. 테스트 데이터에 없는 변수는 0으로 채운다.
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis=0)

# 학습에 사용할 변수
features = []

In [8]:
df.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0,0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0,0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0,0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0,0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
categorical_columns = ['ind_empleado',
                       'pais_residencia',
                       'sexo',
                       'tiprel_1mes',
                       'indresi',
                       'indext',
                       'conyuemp',
                       'canal_entrada',
                       'indfall',
                       'tipodom',
                       'nomprov',
                       'segmento']

In [10]:
# 범주형 변수를 .factorize() 함수를 통해 label encoding한다.
for col in categorical_columns:
    df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_columns

In [11]:
# 수치형 특이값과 결측값을 -99로 대체하고 정수형으로 변환한다.
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)
df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

In [12]:
df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

In [13]:
df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

In [14]:
# 학습에 사용할 수치형 변수를 features에 추구한다.
features += ['age', 'antiguedad', 'renta', 'ind_nuevo', 'indrel', 'indrel_1mes', 'ind_actividad_cliente']

# 2. Feature Engineering

피처 엔지니어링 단계에서는 머신러닝 모델 학습에 사용할 파생 변수를 생성한다. Baseline 모델에서는 전체 24개 고객 변수와 4개의 날짜 변수 기반 파생 변수 그리고 24개의 lag-1 변수를 사용한다.

- 이번 경진대회에서는 N개월 전에 금융 제품을 보유하고 있었는지 여부를 나타내는 Lag 변수가 좋은 파생 변수로 작용한다.
- 1개월 전, 2개월 전, 3개월 전 보유 여부를 현재 고객의 데이터로 이용한다. 
- 하지만 Baseline 모델에서는 1개월 전 정보만을 가져다 사용한다.

In [15]:
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('0')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

In [16]:
# 그 외 변수 결측값은 모두 -99로 대체한다.
df.fillna(-99, inplace=True)

In [17]:
# 날짜를 숫자로 변환한다. 2016-01-28은 1, 2016-06-28은 18로 변환한다.
def date_to_int(str_date): 
    Y, M, D = [int(a) for a in str_date.strip().split("-")]
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

# 날짜를 숫자로 변환하여 int_date에 저장한다.
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

# 데이터를 복사하고, int_date 날짜에 1을 더하여 lag를 생성한다. 변수명에 _prev를 추가한다.
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns]
df_lag['int_date'] += 1

df_trn = df.merge(df_lag, on=['ncodpers', 'int_date'], how='left')
del df, df_lag

In [18]:
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
    
df_trn.fillna(-99, inplace=True)

# lag 1 변수를 추가한다.
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

In [19]:
features

['ind_empleado',
 'pais_residencia',
 'sexo',
 'tiprel_1mes',
 'indresi',
 'indext',
 'conyuemp',
 'canal_entrada',
 'indfall',
 'tipodom',
 'nomprov',
 'segmento',
 'age',
 'antiguedad',
 'renta',
 'ind_nuevo',
 'indrel',
 'indrel_1mes',
 'ind_actividad_cliente',
 'fecha_alta_month',
 'fecha_alta_year',
 'ult_fec_cli_1t_month',
 'ult_fec_cli_1t_year',
 'ind_empleado_prev',
 'pais_residencia_prev',
 'sexo_prev',
 'tiprel_1mes_prev',
 'indresi_prev',
 'indext_prev',
 'conyuemp_prev',
 'canal_entrada_prev',
 'indfall_prev',
 'tipodom_prev',
 'nomprov_prev',
 'segmento_prev',
 'age_prev',
 'antiguedad_prev',
 'renta_prev',
 'ind_nuevo_prev',
 'indrel_prev',
 'indrel_1mes_prev',
 'ind_actividad_cliente_prev',
 'fecha_alta_month_prev',
 'fecha_alta_year_prev',
 'ult_fec_cli_1t_month_prev',
 'ult_fec_cli_1t_year_prev',
 'ind_ahor_fin_ult1_prev',
 'ind_aval_fin_ult1_prev',
 'ind_cco_fin_ult1_prev',
 'ind_cder_fin_ult1_prev',
 'ind_cno_fin_ult1_prev',
 'ind_ctju_fin_ult1_prev',
 'ind_ctma_fin_

# 3. 머신러닝 모델 학습

### 교차 검증

- 경진 대회에서 가장 중요한 것 중의 하나이다.
- 이번 경진 대회에서는 2015-01-28 ~ 2016-05-28, 총 1년 6개월치 데이터가 훈련 데이터로 제공되고 예측해야 하는 테스트 데이터는 2016-06-28 데이터이다.
- 내부 검증 과정에서도 2016-05-28를 검증 데이터로 분리하고 나머지 데이터로 훈련 데이터로 사용하는 것이 일반적이다. Baseline 모델에서는 모델을 간소화하기 위해서 2016-01-28 ~ 2016-04-28 총 4개월치 데이터를 훈련 데이터로 사용하고, 2016-05-28 데이터를 검증 데이터로 사용한다.

In [20]:
# 교차 검증을 위해 데이터 분리하기: 훈련 데이터 전체를 사용하지 않고 2016년도만 사용하도록 추출하는 부분은 피처 엔지니어링에 해당할 수 있다.
# 학습에는 2016-01-28 ~ 2016-04-28 데이터만 사용하고 검증에는 2016-05-28 데이터를 사용한다.
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
del df_trn

In [21]:
# 훈련 데이터에서 신규 구매 건수만 추출한다.
X = []
Y = []

for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    
    X.append(prX)
    Y.append(prY)
    
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

- 제품 변수를 for loop을 돌면서

1) 신규 구매를 한 것에 대해서만 X, Y 변수를 만들었다.

2) 신규 구매를 안 한에 대해서는 만들 수 없는가.

In [22]:
### 훈련, 검증 데이터 분리
vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]

In [23]:
XY_trn.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_valo_fin_ult1_prev,ind_viv_fin_ult1_prev,ind_nomina_ult1_prev,ind_nom_pens_ult1_prev,ind_recibo_ult1_prev,fecha_alta_month_prev,fecha_alta_year_prev,ult_fec_cli_1t_month_prev,ult_fec_cli_1t_year_prev,y
7658069,2016-01-28,1474324,0,0,1,43,2015-10-09,1.0,3,1.0,...,0.0,0.0,0.0,0.0,1.0,10.0,2015.0,0.0,0.0,1
7628180,2016-01-28,1432311,0,0,1,26,2015-08-07,1.0,5,1.0,...,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0,2
7628198,2016-01-28,1432232,0,0,1,33,2015-08-07,0.0,19,1.0,...,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0,2
7628482,2016-01-28,1432080,0,0,0,23,2015-08-07,1.0,5,1.0,...,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0,2
7628692,2016-01-28,1432952,0,0,0,77,2015-08-10,1.0,5,1.0,...,1.0,0.0,0.0,0.0,0.0,8.0,2015.0,0.0,0.0,2


### 모델

#### XGBoost

XGBoost 모델의 파라미터에 대한 설명은 여기에서 확인할 수 있따. 필자가 가장 많이 사용하는 파라미터는 다음과 같다. 필자가 많이 사용하는 파라미터는 다음과 같다.

- max_depth: 트리 모델 최대 깊이를 의미한다. 값이 높을수록 더 복잡한 트리 모델을 생성하며 과적합의 원인이 될 수 있다.
- eta: 딥러닝에서 learning_rate 같은 개념이다. 0과 1사이의 값을 가지며, 값이 너무 높으며 학습이 잘 안 될 수 있다. 반대로 너무 낮으면 학습이 느릴 수 있다.
- colsample_bytree: 트리를 생성할 때 훈련 데이터에서 변수를 샘플링해주는 비율이다. 모든 트리는 전체 변수의 일부만을 학습하여 서로의 약점을 보완해준다. 보통 0.6 ~ 0.9 값을 사용한다.
- colsample_bylevel: 트리의 레벨 별로 훈련 데이터의 변수를 샘플링해주는 비율이다. 보통 0.6 ~ 0.9 값을 사용한다.


- 한 가지 주의해야 할 점이 있다. 머신러닝을 처음 시작하는 분들은 최적 파라미터를 찾는데 너무 많은 시간을 투자한다.

#### 그러나 시간 대비 효율을 생각한다면 파라미터 튜닝보다는 피처 엔지니어링에 더 많은 시간을 쏟을 것을 권장한다.

In [50]:
param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 4,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018
}

In [107]:
# 훈련, 검증 데이터를 Xgboost 형태로 변환한다.
X_trn = XY_trn.as_matrix(columns=features)
Y_trn = XY_trn.as_matrix(columns=['y'])
dtrn = xgb.DMatrix(X_trn, label=Y_trn,feature_names=features)

X_vld = XY_vld.as_matrix(columns=features)
Y_vld = XY_vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

# XGBoost 모델학습
watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)

/anaconda3/envs/base-workspace/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()
/anaconda3/envs/base-workspace/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/anaconda3/envs/base-workspace/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/anaconda3/envs/base-workspace/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


[0]	train-merror:0.423633	eval-merror:0.43452
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 20 rounds.
[1]	train-merror:0.420779	eval-merror:0.433095
[2]	train-merror:0.404525	eval-merror:0.418001
[3]	train-merror:0.402465	eval-merror:0.41597
[4]	train-merror:0.397556	eval-merror:0.410032
[5]	train-merror:0.394621	eval-merror:0.406206
[6]	train-merror:0.394211	eval-merror:0.40647
[7]	train-merror:0.393057	eval-merror:0.406206
[8]	train-merror:0.392387	eval-merror:0.405283
[9]	train-merror:0.392213	eval-merror:0.405705
[10]	train-merror:0.391915	eval-merror:0.405705
[11]	train-merror:0.391698	eval-merror:0.405362
[12]	train-merror:0.391629	eval-merror:0.405124
[13]	train-merror:0.391158	eval-merror:0.404702
[14]	train-merror:0.390972	eval-merror:0.404544
[15]	train-merror:0.390711	eval-merror:0.404386
[16]	train-merror:0.390407	eval-merror:0.404043
[17]	train-merror:0.390283	eval-merror:0.404069
[1

In [108]:
import pickle
pickle.dump(model, open('../santander/xgb.baseline.pkl', "wb"))
best_ntree_limit = model.best_ntree_limit

In [60]:
import pickle
with open('../santander/xgb.baseline.pkl', 'rb') as f:
     model = pickle.load(f)

In [62]:
best_ntree_limit = model.best_ntree_limit
best_ntree_limit

108

### 교차 검증

교차 검증에서는 이번 경진대회의 평가척도인 MAP@7을 사용하여 성능 수준을 확인한다.

In [26]:
# MAP@7 평가 척도를 위한 준비작업이다.
vld = trn[trn['fecha_dato'] == vld_date]
ncodpers_vld = vld.as_matrix(columns=['ncodpers'])

# 검증 데이터에서 신규 구매를 구한다.
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    
    # 신규 구매
    vld[padd] = vld[prod] - vld[prev]
    
add_vld = vld.as_matrix(columns=[prod + '_add' for prod in prods])
add_vld_list = [list() for i in range(len(ncodpers_vld))]

/anaconda3/envs/base-workspace/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/envs/base-workspace/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/envs/base-workspace/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


In [27]:
# 고객별 신규 구매 정답값을 add_vld_list에 저장하고, 총 count를 count_vld에 저장한다.
count_vld = 0

for ncodper in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodper, prod] > 0:
            add_vld_list[ncodper].append(prod)
            count_vld += 1            

In [28]:
from mapk import mapk

In [29]:
print(mapk(add_vld_list, add_vld_list, 7, 0.0))

0.04266379915553903


In [31]:
# 검증 데이터에 대한 예측 값
X_vld = vld.as_matrix(columns=features)
Y_vld = vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)
preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)

/anaconda3/envs/base-workspace/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/anaconda3/envs/base-workspace/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
# 저번 달에 보유한 신규 구매가 불가하기 때문에 확률 값에서 1을 미리 뺴준다.
preds_vld = preds_vld - vld.as_matrix(columns=[prod + '_prev' for prod in prods])

/anaconda3/envs/base-workspace/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [34]:
# 검증 데이터 예측 상위 7개
result_vld = []

for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods = [(y, p, ip) for y, p, ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result_vld.append([ip for y, p, ip in y_prods])

In [70]:
y_prods

[(0.7294137477874756, 'ind_cco_fin_ult1', 2),
 (0.1431470513343811, 'ind_ctma_fin_ult1', 6),
 (0.03608470410108566, 'ind_recibo_ult1', 23),
 (0.030514413490891457, 'ind_nomina_ult1', 21),
 (0.030175069347023964, 'ind_nom_pens_ult1', 22),
 (0.02091251127421856, 'ind_cno_fin_ult1', 4),
 (0.0037921576295048, 'ind_ecue_fin_ult1', 12)]

In [35]:
# 검증 데이터에서의 MAP 점수
print(mapk(add_vld_list, result_vld, 7, 0.0))

0.036440017704771004


In [44]:
result_vld[:20]

[[23, 18, 12, 7, 22, 21, 11],
 [23, 18, 12, 22, 21, 11, 4],
 [17, 7, 2, 8, 19, 11, 15],
 [23, 18, 12, 21, 22, 7, 13],
 [23, 12, 22, 4, 21, 19, 13],
 [23, 12, 17, 7, 22, 21, 4],
 [23, 12, 17, 22, 21, 7, 4],
 [2, 12, 18, 23, 4, 22, 13],
 [23, 12, 17, 4, 22, 21, 8],
 [23, 21, 22, 18, 12, 2, 7],
 [2, 17, 12, 23, 22, 4, 7],
 [17, 23, 12, 21, 22, 7, 4],
 [23, 17, 12, 22, 21, 7, 4],
 [23, 18, 12, 22, 21, 4, 11],
 [23, 18, 22, 12, 21, 4, 17],
 [23, 18, 22, 21, 11, 4, 17],
 [23, 18, 12, 21, 22, 4, 19],
 [12, 23, 17, 7, 22, 4, 8],
 [23, 18, 12, 11, 22, 21, 19],
 [2, 23, 22, 21, 4, 18, 19]]

In [45]:
add_vld_list[:20]

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [23],
 [],
 [],
 []]

# 4. 테스트 데이터 예측 및 캐글 업로드

- 테스트 데이터에 대해서 조금이라도 좋은 성능을 내기 위하여 훈련 데이터와 검증 데이터를 합한 전체 데이터에 대해서 XGBoost를 다시 학습한다.

In [63]:
# XGBoost 모델을 전체 훈련 데이터로 재학습한다.
X_all = XY.as_matrix(columns=features)
Y_all = XY.as_matrix(columns=['y'])
dall = xgb.DMatrix(X_all, label=Y_all, feature_names=features)

/anaconda3/envs/base-workspace/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/anaconda3/envs/base-workspace/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [64]:
watch_list = [(dall, 'train')]

# 트리 개수를 늘어난 데이터 양만큼 비례해서 증가한다.
best_ntree_limit = int(best_ntree_limit * (len(XY_trn) + len(XY_vld)) / len(XY_trn))
best_ntree_limit

133

In [65]:
model = xgb.train(param, dall, num_boost_round=best_ntree_limit, evals=watch_list, early_stopping_rounds=20)

[0]	train-merror:0.418521
Will train until train-merror hasn't improved in 20 rounds.
[1]	train-merror:0.401112
[2]	train-merror:0.398681
[3]	train-merror:0.396797
[4]	train-merror:0.396279
[5]	train-merror:0.396199
[6]	train-merror:0.395832
[7]	train-merror:0.395651
[8]	train-merror:0.395113
[9]	train-merror:0.394415
[10]	train-merror:0.394063
[11]	train-merror:0.393692
[12]	train-merror:0.393656
[13]	train-merror:0.393375
[14]	train-merror:0.393189
[15]	train-merror:0.392983
[16]	train-merror:0.392651
[17]	train-merror:0.392461
[18]	train-merror:0.39239
[19]	train-merror:0.392139
[20]	train-merror:0.391802
[21]	train-merror:0.391576
[22]	train-merror:0.391375
[23]	train-merror:0.39127
[24]	train-merror:0.391079
[25]	train-merror:0.390818
[26]	train-merror:0.390692
[27]	train-merror:0.390571
[28]	train-merror:0.390521
[29]	train-merror:0.390441
[30]	train-merror:0.39021
[31]	train-merror:0.390104
[32]	train-merror:0.389853
[33]	train-merror:0.389717
[34]	train-merror:0.389622
[35]	tra

In [66]:
# 변수 중요도를 출력한다.
print("Feature importance: ")

for kv in sorted([(k, v) for k, v in model.get_fscore().items()], key=lambda kv:kv[1], reverse=True):
    print(kv)

Feature importance: 
('age', 11489)
('renta', 10479)
('antiguedad', 10351)
('age_prev', 7197)
('antiguedad_prev', 6901)
('fecha_alta_month', 6685)
('nomprov', 6581)
('fecha_alta_year', 5972)
('canal_entrada', 4582)
('renta_prev', 3928)
('nomprov_prev', 3521)
('canal_entrada_prev', 2549)
('ind_ecue_fin_ult1_prev', 2352)
('ind_recibo_ult1_prev', 2287)
('ind_cco_fin_ult1_prev', 2122)
('fecha_alta_month_prev', 2116)
('sexo', 2054)
('ind_cno_fin_ult1_prev', 1929)
('fecha_alta_year_prev', 1862)
('ind_tjcr_fin_ult1_prev', 1486)
('ind_reca_fin_ult1_prev', 1404)
('segmento', 1362)
('ind_nom_pens_ult1_prev', 1344)
('ind_nomina_ult1_prev', 1295)
('segmento_prev', 1254)
('tiprel_1mes', 1153)
('ind_valo_fin_ult1_prev', 1139)
('ind_actividad_cliente', 1107)
('ind_dela_fin_ult1_prev', 997)
('ind_ctop_fin_ult1_prev', 911)
('tiprel_1mes_prev', 822)
('sexo_prev', 815)
('ind_ctma_fin_ult1_prev', 758)
('ind_fond_fin_ult1_prev', 717)
('ind_ctpp_fin_ult1_prev', 702)
('indext', 592)
('ind_actividad_cliente_p

In [71]:
# 캐글 제출을 위하여 테스트 데이터에 대한 예측 값을 구한다.
X_tst = tst.as_matrix(columns=features)
dtst = xgb.DMatrix(X_tst, feature_names=features)
preds_tst = model.predict(dtst, ntree_limit=best_ntree_limit)
ncodpers_tst = tst.as_matrix(columns=['ncodpers'])

# 신규만 생성한다.
preds_tst = preds_tst - tst.as_matrix(columns=[prod + '_prev' for prod in prods])

/anaconda3/envs/base-workspace/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/anaconda3/envs/base-workspace/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
/anaconda3/envs/base-workspace/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [85]:
submit_file = open('./xgb.baseline.2015-06-28', 'w')
submit_file.write('ncodpers,added_products\n')

for ncodper, pred in zip(ncodpers_tst, preds_tst):
    y_prods = [(y, p, ip) for y, p, ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    y_prods = [p for y, p, ip in y_prods]
    submit_file.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))

In [86]:
len(ncodpers_tst), len(preds_tst)

(929615, 929615)

In [87]:
lst = []

f = open('./xgb.baseline.2015-06-28', 'r')
while True:
    line = f.readline()
    if not line: 
        break
    lst.append(line)
f.close()

In [91]:
len(lst)

929616